In [3]:
# install from PyPI
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00


In [5]:
!pip install apify-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 2.8 MB/s eta 0:00:00


In [161]:
def get_user_inputs():
    enterprise_name = input("Enterprise Name to search for: ")
    keywords = input("Keyword(s) to search for: ")
    insight_request = input("Type of insight to ask about the enterprise's data: ")
    specific_websites = input("(Optional) Specific websites to scrape using available Apify actors or a separate website URL: ")
    return enterprise_name, keywords, insight_request, specific_websites


In [166]:
import pandas as pd

In [169]:
import openai

def expand_inputs(enterprise_name, keywords):
    openai.api_key = 'your-api'

    # response = openai.chat.completions.create(
    # model="gpt-3.5-turbo",
    #  messages=[
    #     {"role": "system", "content": "You are an assistant skilled in generating common spelling errors and typos."},
    #     {"role": "user", "content": f"Generate common spelling errors and typos for the following:\n\nEnterprise Name: {enterprise_name}\nKeyword(s): {keywords}"}
    # ],
    # max_tokens=100)

    # variations = response.choices[0].text.strip().split('\n')
    # enterprise_variations = [v for v in variations if "Enterprise Name" in v]
    # keyword_variations = [v for v in variations if "Keyword(s)" in v]
    enterprise_variations =["Targt",
    "Targte",
    "Taregt"]
    keyword_variations = ["babby",
    "babt",
    "produts",
    "prodacts"]
    enterprise_variations.append(enterprise_name)
    keyword_variations.extend(keywords)

    enterprise_variations = [str(word).lower() for word in enterprise_variations]
    keyword_variations = [str(word).lower() for word in  keyword_variations]

    return enterprise_variations, keyword_variations


In [170]:
from apify_client import ApifyClient

def scrape_data(enterprise_variations, keyword_variations, specific_websites=None):
    client = ApifyClient('your-apify-api')

    search_queries = [[ev, kv] for ev in enterprise_variations for kv in keyword_variations]  #generating combinations

    results = []



    if specific_websites:
        for website in specific_websites.split(','):
            for query in search_queries:


                run_input = {
                    "hashtags": query,
                    "resultsLimit": 3,
                }
                run = client.actor("apify/instagram-hashtag-scraper").call(run_input=run_input)
                for item in client.dataset(run["defaultDatasetId"]).iterate_items():
                  results.append(item)

    return results


In [171]:
def filter_rows(row, en, prd):

    if isinstance(row, list):  # Check if row is a list
        hashtags = [tag.lower() for tag in row]

        found_list1 = any(word in hashtags for word in en) #ent
        found_list2 = any(word in hashtags for word in prd) #key
        # print(found_list1, found_list2)
        if found_list1 and found_list2:
          return True

        return False

    else:
        return False  # Return False if row is not a list


In [172]:
def analyze_data(results, insight_request):
    openai.api_key = 'you-key'
    context = f"Data: {results}\nInsight Request: {insight_request}"

    # response = openai.chat.completions.create(
    #   model="gpt-3.5-turbo",
    #    messages=[
    #     {"role": "system", "content": "You are an assistant skilled in providing analytical insights of any data."},
    #     {"role": "user", "content": context}
    # ],
    #   max_tokens=300
    # )

    # return response.choices[0].text.strip()

    return "Successfull"


In [175]:
def main():
    enterprise_name, keywords, insight_request, specific_websites = get_user_inputs()
    keywords = keywords.split(',')
    keywords = [word.strip() for keyword in keywords for word in keyword.split()]

    enterprise_variations, keyword_variations = expand_inputs(enterprise_name, keywords)
    print("Inputs received")


    print("Scraping data")
    #[targte, bby]

    results = scrape_data(enterprise_variations, keyword_variations, specific_websites)
    df = pd.DataFrame(results)


    desired_columns = ['caption', 'commentsCount','hashtags','isSponsored']
    selected_df = df[desired_columns]
    selected_df = selected_df.drop(df.index[0])
    filtered_rows = selected_df[selected_df['hashtags'].apply(filter_rows, args=(enterprise_variations, keyword_variations))]
    json_data = filtered_rows .to_json(orient='records')

    print(json_data)


    insights = analyze_data(json_data, insight_request)
    print("Generated Insights:\n", insights)

if __name__ == "__main__":
    main()


Enterprise Name to search for: Target
Keyword(s) to search for: Baby Products
Type of insight to ask about the enterprise's data: For the majority of reviews, do people like or dislike Target's baby products? Please provide sources.
(Optional) Specific websites to scrape using available Apify actors or a separate website URL: https://www/instagram.com/
Inputs received
Scraping data
[{"caption":"Sophi con este hermoso Ballon Romper de  @target \ud83c\udf38 y su mu\u00f1equita de #leyadoll_com \ud83d\udc96\n\n#targetstyle #target #catandjackfortarget #babyclothing #babyclothes #baby #babygirl","commentsCount":0,"hashtags":["leyadoll_com","targetstyle","target","catandjackfortarget","babyclothing","babyclothes","baby","babygirl"],"isSponsored":false},{"caption":"Sophi con este hermoso Ballon Romper de  @target \ud83c\udf38 y su mu\u00f1equita de #leyadoll_com \ud83d\udc96\n\n#targetstyle #target #catandjackfortarget #babyclothing #babyclothes #baby #babygirl","commentsCount":0,"hashtags":